In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13942322116408963919, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 4522463157089883214
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:83:00.0, compute capability: 6.1", name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 4647208257778083951
 physical_device_desc: "device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:84:00.0, compute capability: 6.1", name: "/device:GPU:2"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 9573010081830452690
 physical_device_desc: "device: 2, name: GeForce GTX 1080 Ti, pci bus id: 0000:87:00.0, compute capability: 6.1", name: "/device:GPU:3"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 17028737498273194625
 physical_devi

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time

import os
import pandas as pd

import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize

import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.4.0


In [5]:
output_classes = 2
batch_size = 64
img_height, img_width = 224, 224
input_shape = (img_height, img_width, 3)
epochs = 50

nb_train_samples = 103104
nb_validation_samples = 12288
nb_test_samples = 697

In [6]:
train_dir = 'data/train/'
validation_dir = 'data/validation'
test_dir = 'data/test'

In [7]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = True,
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = True,
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 103104 images belonging to 2 classes.
Found 12288 images belonging to 2 classes.
Found 697 images belonging to 2 classes.


In [8]:
def vgg19_model():
    base_model = VGG19(weights='imagenet', include_top=False, pooling='avg')
    x = base_model.output
    # x = GlobalAveragePooling2D()(x)
    x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.5)(x)
    prediction = Dense(output_classes, activation=tf.nn.softmax)(x)

    model = Model(inputs=base_model.input,outputs=prediction)
    return model

In [9]:
space = {
         'lr': hp.choice('lr',[0.001, 0.0001, 0.00001,0.000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'beta_1':hp.choice('beta_1',[0.3,0.4,0.5,0.6,0.7,0.8]),
#          'beta_2':hp.choice('beta_2',[0.99,0.995,0.7,0.8,0.9,0.999]),
         'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
         'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [11]:
def f_nn(params):   
    print ('Parameters testing: ', params)
#     batch_size=params['batch_size']

#     random_seed = np.random.seed(1132)

#     train_datagen = ImageDataGenerator(
#         rescale=1. / 255,
#         featurewise_center=True,
#         featurewise_std_normalization=True)

#     test_datagen = ImageDataGenerator(rescale=1. / 255)

#     train_generator = train_datagen.flow_from_directory(
#         train_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         shuffle = True,
#         seed = random_seed,
#         class_mode='categorical')

#     validation_generator = train_datagen.flow_from_directory(
#         validation_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         shuffle = True,
#         seed = random_seed,
#         class_mode='categorical')

#     test_generator = test_datagen.flow_from_directory(
#         test_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         shuffle = False,
#         class_mode='categorical')

   
#     adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0, amsgrad=params['amsgrad'])
    sgd_opt=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = vgg19_model()
    
    model.compile(optimizer = sgd_opt, loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit_generator(
      train_generator,
      steps_per_epoch = nb_train_samples // batch_size,
      epochs = 15,
      validation_data = validation_generator,
      validation_steps = nb_validation_samples // batch_size,
      verbose = 1)
    
    score = model.evaluate_generator(validation_generator, steps=50)

    print ('Validation Score: ', score[0])
    print ('Validation Accuracy: ',score[1])
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    predicts = model.predict_generator(test_generator, steps=test_generator.samples/test_generator.batch_size, verbose=1)
    predict_class = np.argmax(predicts, axis=1)
    errors = np.where(predict_class != truth)[0]
    print("No of errors = {}/{}".format(len(errors),test_generator.samples))
    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': score[0], 'status': STATUS_OK, 'model': model}

In [12]:
# def get_callbacks(params):
#     callbacks =[EarlyStopping(monitor='val_acc', patience=5, verbose=1),
#                 ModelCheckpoint('callbacks/{}.h5'.format(params['batch_size']), save_best_only=True),
#              TensorBoard('tensorlogs/logs-gridsearch', write_graph=True, write_grads=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)]
#     return callbacks

In [13]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=30, trials=trials)
print(best)

Parameters testing:  {'lr': 0.001, 'momentum': 0.3, 'nesterov': False}
Epoch 1/15
1611/1611 [==============================] - 1206s 749ms/step - loss: 0.4097 - acc: 0.9042 - val_loss: 0.2938 - val_acc: 0.9461
Epoch 2/15
1611/1611 [==============================] - 1210s 751ms/step - loss: 0.2606 - acc: 0.9659 - val_loss: 0.3297 - val_acc: 0.9358
Epoch 3/15
1611/1611 [==============================] - 1212s 752ms/step - loss: 0.2208 - acc: 0.9816 - val_loss: 0.7393 - val_acc: 0.8145
Epoch 4/15
1611/1611 [==============================] - 1209s 751ms/step - loss: 0.2039 - acc: 0.9882 - val_loss: 0.2722 - val_acc: 0.9644
Epoch 5/15
1611/1611 [==============================] - 1205s 748ms/step - loss: 0.1914 - acc: 0.9927 - val_loss: 0.3810 - val_acc: 0.9390
Epoch 6/15
1611/1611 [==============================] - 1205s 748ms/step - loss: 0.1850 - acc: 0.9950 - val_loss: 0.2826 - val_acc: 0.9674
Epoch 7/15
1611/1611 [==============================] - 1209s 750ms/step - loss: 0.1817 - acc: 

1611/1611 [==============================] - 1199s 744ms/step - loss: 0.4764 - acc: 0.8847 - val_loss: 0.5946 - val_acc: 0.8737
Epoch 7/15
1611/1611 [==============================] - 1190s 739ms/step - loss: 0.4889 - acc: 0.8847 - val_loss: 0.4933 - val_acc: 0.8845
Epoch 8/15
1611/1611 [==============================] - 1193s 740ms/step - loss: 0.4779 - acc: 0.8956 - val_loss: 0.4762 - val_acc: 0.8961
Epoch 9/15
1611/1611 [==============================] - 1193s 740ms/step - loss: 0.4861 - acc: 0.8968 - val_loss: 0.5292 - val_acc: 0.8735
Epoch 10/15
1611/1611 [==============================] - 1193s 740ms/step - loss: 0.4866 - acc: 0.9019 - val_loss: 0.5027 - val_acc: 0.8923
Epoch 11/15
1611/1611 [==============================] - 1197s 743ms/step - loss: 0.4938 - acc: 0.9026 - val_loss: 0.5059 - val_acc: 0.8817
Epoch 12/15
1611/1611 [==============================] - 1188s 737ms/step - loss: 0.4944 - acc: 0.9066 - val_loss: 0.5489 - val_acc: 0.8843
Epoch 13/15
1611/1611 [============

Epoch 12/15
1611/1611 [==============================] - 1222s 759ms/step - loss: 0.1745 - acc: 0.9982 - val_loss: 0.2751 - val_acc: 0.9694
Epoch 13/15
1611/1611 [==============================] - 1217s 755ms/step - loss: 0.1714 - acc: 0.9993 - val_loss: 0.2460 - val_acc: 0.9788
Epoch 14/15
1611/1611 [==============================] - 1220s 757ms/step - loss: 0.1731 - acc: 0.9986 - val_loss: 0.2596 - val_acc: 0.9735
Epoch 15/15
1611/1611 [==============================] - 1222s 759ms/step - loss: 0.1724 - acc: 0.9988 - val_loss: 0.2700 - val_acc: 0.9711
Validation Score:  0.23509732156991958
Validation Accuracy:  0.97875
11/10 [==============================] - 10s 889ms/step
No of errors = 20/697
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'lr': 0.0001, 'momentum': 0.9, 'nesterov': False}
Epoch 1/15
1611/1611 [==============================] - 1211s 752ms/st

Parameters testing:  {'lr': 0.0001, 'momentum': 0.5, 'nesterov': False}
Epoch 1/15
1611/1611 [==============================] - 1214s 753ms/step - loss: 0.5649 - acc: 0.8416 - val_loss: 0.5167 - val_acc: 0.8734
Epoch 2/15
1611/1611 [==============================] - 1221s 758ms/step - loss: 0.3956 - acc: 0.9161 - val_loss: 0.3452 - val_acc: 0.9268
Epoch 3/15
1611/1611 [==============================] - 1216s 755ms/step - loss: 0.3410 - acc: 0.9341 - val_loss: 0.3073 - val_acc: 0.9446
Epoch 4/15
1611/1611 [==============================] - 1210s 751ms/step - loss: 0.3080 - acc: 0.9479 - val_loss: 0.3065 - val_acc: 0.9403
Epoch 5/15
1611/1611 [==============================] - 1208s 750ms/step - loss: 0.2823 - acc: 0.9579 - val_loss: 0.3301 - val_acc: 0.9361
Epoch 6/15
1611/1611 [==============================] - 1202s 746ms/step - loss: 0.2650 - acc: 0.9639 - val_loss: 0.2846 - val_acc: 0.9556
Epoch 7/15
1611/1611 [==============================] - 1217s 756ms/step - loss: 0.2496 - acc:

1611/1611 [==============================] - 1211s 752ms/step - loss: 0.1798 - acc: 0.9964 - val_loss: 0.2855 - val_acc: 0.9692
Epoch 7/15
1611/1611 [==============================] - 1209s 751ms/step - loss: 0.1762 - acc: 0.9976 - val_loss: 0.4005 - val_acc: 0.9507
Epoch 8/15
1611/1611 [==============================] - 1214s 753ms/step - loss: 0.1715 - acc: 0.9990 - val_loss: 0.2554 - val_acc: 0.9797
Epoch 9/15
1611/1611 [==============================] - 1206s 748ms/step - loss: 0.1724 - acc: 0.9987 - val_loss: 0.2423 - val_acc: 0.9728
Epoch 10/15
1611/1611 [==============================] - 1216s 755ms/step - loss: 0.1704 - acc: 0.9991 - val_loss: 0.2306 - val_acc: 0.9796
Epoch 11/15
1611/1611 [==============================] - 1209s 750ms/step - loss: 0.1716 - acc: 0.9985 - val_loss: 0.2470 - val_acc: 0.9797
Epoch 12/15
1611/1611 [==============================] - 1214s 754ms/step - loss: 0.1686 - acc: 0.9995 - val_loss: 0.3588 - val_acc: 0.9504
Epoch 13/15
1611/1611 [============

Epoch 12/15
1611/1611 [==============================] - 1215s 754ms/step - loss: 0.1898 - acc: 0.9937 - val_loss: 0.2923 - val_acc: 0.9608
Epoch 13/15
1611/1611 [==============================] - 1214s 754ms/step - loss: 0.1886 - acc: 0.9939 - val_loss: 0.2513 - val_acc: 0.9704
Epoch 14/15
1611/1611 [==============================] - 1213s 753ms/step - loss: 0.1861 - acc: 0.9950 - val_loss: 0.2505 - val_acc: 0.9699
Epoch 15/15
1611/1611 [==============================] - 1214s 754ms/step - loss: 0.1842 - acc: 0.9955 - val_loss: 0.2715 - val_acc: 0.9663
Validation Score:  0.2663200092315674
Validation Accuracy:  0.9671875
11/10 [==============================] - 9s 801ms/step
No of errors = 25/697
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'lr': 1e-05, 'momentum': 0.7, 'nesterov': False}
Epoch 1/15
1611/1611 [==============================] - 1209s 751ms/ste

Parameters testing:  {'lr': 0.001, 'momentum': 0.7, 'nesterov': False}
Epoch 1/15
1611/1611 [==============================] - 1212s 752ms/step - loss: 0.4000 - acc: 0.9110 - val_loss: 0.2995 - val_acc: 0.9468
Epoch 2/15
1611/1611 [==============================] - 1214s 754ms/step - loss: 0.2554 - acc: 0.9690 - val_loss: 0.5165 - val_acc: 0.8857
Epoch 3/15
1611/1611 [==============================] - 1207s 749ms/step - loss: 0.2132 - acc: 0.9842 - val_loss: 0.2443 - val_acc: 0.9716
Epoch 4/15
1611/1611 [==============================] - 1209s 750ms/step - loss: 0.1931 - acc: 0.9920 - val_loss: 0.2558 - val_acc: 0.9705
Epoch 5/15
1611/1611 [==============================] - 1222s 758ms/step - loss: 0.1878 - acc: 0.9938 - val_loss: 0.2646 - val_acc: 0.9704
Epoch 6/15
1611/1611 [==============================] - 1217s 755ms/step - loss: 0.1779 - acc: 0.9972 - val_loss: 0.2512 - val_acc: 0.9724
Epoch 7/15
1611/1611 [==============================] - 1206s 749ms/step - loss: 0.1751 - acc: 

ResourceExhaustedError: OOM when allocating tensor with shape[64,64,112,112]
	 [[Node: training_21/SGD/gradients/block1_pool_21/MaxPool_grad/MaxPoolGrad = MaxPoolGrad[T=DT_FLOAT, _class=["loc:@block1_pool_21/MaxPool"], data_format="NHWC", ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1], _device="/job:localhost/replica:0/task:0/device:GPU:0"](block1_conv2_21/Relu, block1_pool_21/MaxPool, training_21/SGD/gradients/block2_conv1_21/convolution_grad/Conv2DBackpropInput)]]

Caused by op 'training_21/SGD/gradients/block1_pool_21/MaxPool_grad/MaxPoolGrad', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Data/home/sah623/venv/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Data/home/sah623/venv/lib/python3.6/site-packages/traitlets/config/application.py", line 658, i